In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import tensorflow as tf
import dask.array as da
# from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Concatenate, Conv2DTranspose, MaxPool2D, Flatten, Conv2D, Reshape, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from keras.callbacks import Callback
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import LeakyReLU, BatchNormalization, UpSampling2D, concatenate, Activation
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import load_model
# from keras.saving import register_keras_serializable
from tensorflow.keras.utils import register_keras_serializable

from tensorflow.keras.layers import GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Conv2DTranspose, Input

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Dropout, Concatenate, LeakyReLU, Add
from tensorflow.keras.initializers import he_normal

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import cm 
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
import matplotlib.colors as mcolors
from matplotlib.colors import BoundaryNorm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
plt.style.use("dark_background")

# from sklearn.preprocessing import MaxAbsScaler
# from sklearn.preprocessing import MinMaxScaler

# import properscoring as ps
import xskillscore as xs

2024-08-29 15:47:00.224284: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-29 15:47:00.224334: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-29 15:47:00.225457: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-29 15:47:00.243444: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-29 15:47:02.707677: W tensorflow/compiler/tf2

In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  4


In [3]:
# Get the list of available physical GPUs
gpus = tf.config.list_physical_devices('GPU')

# Set memory growth for each GPU
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Create MirroredStrategy
# strategy = tf.distribute.MirroredStrategy()
# strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0"])
# strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
# strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1", "/gpu:2"])
strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1", "/gpu:2", "/gpu:3" ])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


2024-08-29 15:47:11.484623: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12058 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:03:00.0, compute capability: 8.0
2024-08-29 15:47:11.487250: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 12164 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:44:00.0, compute capability: 8.0
2024-08-29 15:47:11.489743: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 12164 MB memory:  -> device: 2, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:84:00.0, compute capability: 8.0
2024-08-29 15:47:11.492270: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 12188 MB memory:  -> device: 3, name: NVIDIA A100-SXM4-80GB, pci bu

In [4]:
diri = '/work/bb0983/athul_satheesh/e_obs_precip/'
diro = '/work/bb0983/athul_satheesh/downscaled_data/europe/figures/'

coarse_raw = 'rr_ens_mean_1.0deg_reg_v29.0e.nc'
fine_raw = 'rr_ens_mean_0.1deg_reg_v29.0e.nc'

lati = 43#40
latf = 59#60

loni = -6#-10
lonf = 15#30

# test period
strt = '2001-01-01'
last = '2023-12-31'

coarse_data = xr.open_dataset(diri+coarse_raw).rr.transpose('time','lat','lon').sel(time=slice(strt, last), 
                                                                                    lat=slice(lati, latf), 
                                                                                    lon=slice(loni, lonf)
                                                                                   )
fine_data = xr.open_dataset(diri+fine_raw).rr.transpose('time','latitude','longitude').sel(time=slice(strt, last), 
                                                                                           latitude=slice(lati, latf), 
                                                                                           longitude=slice(loni, lonf)
                                                                                          )
fine_data = fine_data.rename({'latitude':'lat', 'longitude':'lon'})

In [5]:
dims_coarse = coarse_data.shape# def build_generator(input_shape):
#     inputs = Input(shape=input_shape, name='generator_input')

#     # Convolutions
#     conv_4 = conv_block(inputs, 16, (3, 3), 'conv_4', (1,1))
#     conv_3 = conv_block(conv_4, 32, (3, 3), 'conv_3', (1,1))
#     conv_2 = conv_block(conv_3, 64, (3, 3), 'conv_2', (1,1))
#     conv_1 = conv_block(conv_2, 128, (3, 3), 'conv_1', (1,1))
#     conv_0 = conv_block(conv_1, 256, (3, 3), 'conv_0', (1,1))
#     conv_00 = conv_block(conv_0, 512, (3, 3), 'conv_00', (1,1))
    
#     # Upsampling
#     deconv_00 = deconv_block(conv_00, 512, (3, 3), (1,1), 'deconv_00', (1,1))
#     deconv_0 = deconv_block(deconv_00, 256, (3, 3), (1,1), 'deconv_0', (1,1))
#     deconv_1 = deconv_block(deconv_0, 128, (3, 3), (1,1), 'deconv_1', (1,1))
#     deconv_2 = deconv_block(deconv_1, 64, (3, 3), (1,1), 'deconv_2', (1,1))
#     # deconv_3 = deconv_block(deconv_2, 32, (3, 3), (2,2), 'deconv_3', (1,1))
#     deconv_3 = deconv_block(deconv_2, 32, (3, 3), (1,1), 'deconv_3', (1,1))
#     deconv_4 = deconv_block(deconv_3, 16, (6, 6), (5,5), 'deconv_4', (1,1))
    
#     # Final Upsample to required shape
#     # outputs = Conv2D(filters=1, kernel_size=(3, 3), kernel_initializer=he_normal(), padding='same', 
#     #                           strides=(1, 1), activation='relu', name='final_conv')(deconv_4)
#     outputs = Conv2DTranspose(filters=1, kernel_size=(3, 3), kernel_initializer=he_normal(), padding='same', 
#                               strides=(2, 2), activation='relu', name='final_deconv')(deconv_4)
    
#     model = Model(inputs=inputs, outputs=outputs, name='generator')
    
#     return model
dims_fine = fine_data.shape

In [6]:
coarse_nan_mask = np.isnan(coarse_data)
fine_nan_mask = np.isnan(fine_data)

fill_val = -1e-10

In [7]:
coarse_data = coarse_data.fillna(fill_val)
fine_data = fine_data.fillna(fill_val)

In [8]:
epoch = 1800 #1450 #650 #500 #1500 #650 

In [9]:
# @register_keras_serializable()
# class RandomDropout(tf.keras.layers.Dropout):
#     def __init__(self, min_rate=0.25, max_rate=0.35, **kwargs):
#         # Pass a dummy rate (e.g., 0.0) to satisfy the base class constructor
#         super(RandomDropout, self).__init__(rate=0.0, **kwargs)
#         self.min_rate = min_rate
#         self.max_rate = max_rate

#     def call(self, inputs, training=None):
#         # Apply dropout with a random rate between min_rate and max_rate
#         self.rate = np.random.uniform(self.min_rate, self.max_rate)
#         return super(RandomDropout, self).call(inputs, training=True)  # Force dropout always

#     def get_config(self):
#         config = super(RandomDropout, self).get_config()
#         config.update({
#             'min_rate': self.min_rate,
#             'max_rate': self.max_rate,
#         })
#         return config

In [10]:
model_diri = '/work/bb0983/athul_satheesh/downscaled_data/europe/trained_models/wgan_gp_extended_eu/' #'/work/bb0983/athul_satheesh/downscaled_data/europe/trained_models/wgan_gp/'
model_fili = f'generator_prob_epoch_{epoch}_adamV5.keras'
model_fili_dis = f'discriminator_prob_epoch_{epoch}_adamV5.keras'

In [11]:
# # Use a custom dropout layer that is active also during prediction and use this in the model instead of the default Dropout.
# # Alternatively you can also set "training=True" in the Dropout layer
# @register_keras_serializable()
# class CustomDropout(Dropout):
#     def __init__(self, rate, **kwargs):
#         super(CustomDropout, self).__init__(rate, **kwargs)

#     def call(self, inputs, training=None):
#         return super().call(inputs, training=True)  # Always active

In [12]:
with strategy.scope():
    # Load the generator 
    generator = load_model(model_diri+model_fili)
    discriminator = load_model(model_diri+model_fili_dis)

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [13]:
generator.summary()

Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 generator_input (InputLaye  [(None, 16, 21, 1)]       0         
 r)                                                              
                                                                 
 conv_4_conv (Conv2D)        (None, 16, 21, 16)        160       
                                                                 
 conv_4_lrelu (LeakyReLU)    (None, 16, 21, 16)        0         
                                                                 
 conv_4_bn (BatchNormalizat  (None, 16, 21, 16)        64        
 ion)                                                            
                                                                 
 conv_4_dropout (Dropout)    (None, 16, 21, 16)        0         
                                                                 
 conv_3_conv (Conv2D)        (None, 16, 21, 32)        46

In [14]:
discriminator.summary()

Model: "wGAN_discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 discriminator_input (Input  [(None, 160, 210, 1)]     0         
 Layer)                                                          
                                                                 
 conv1_conv (Conv2D)         (None, 80, 105, 128)      1280      
                                                                 
 conv1_lrelu (LeakyReLU)     (None, 80, 105, 128)      0         
                                                                 
 conv1_bn (BatchNormalizati  (None, 80, 105, 128)      512       
 on)                                                             
                                                                 
 conv3_conv (Conv2D)         (None, 80, 105, 256)      295168    
                                                                 
 conv3_lrelu (LeakyReLU)     (None, 80, 105, 256

In [15]:
def generate_downscaled_data(generator, coarse_data, fine_data, fine_nan_mask, num_members=10):
    downscaled_data_list = []
    
    for _ in range(num_members):
        # Generate noise
        # noise_factor = 10
        # noisy_coarse_data = coarse_data + noise_factor * tf.random.normal(shape=coarse_data.shape, mean=0.2, stddev=0.05)
        # noisy_coarse_data = xr.where(noisy_coarse_data<0, 0.0, noisy_coarse_data)
        
        # Generate predictions
        downscaled_data = generator.predict(coarse_data.values.reshape(coarse_data.shape + (1,)), batch_size=32)
        # downscaled_data = generator.predict(noisy_coarse_data.values.reshape(noisy_coarse_data.shape + (1,)), batch_size=32)
        
        # Create DataArray with proper attributes
        downscaled_data_da = xr.DataArray(
            name='precipitation', 
            data=downscaled_data.squeeze(), 
            dims=fine_data.dims, 
            coords=fine_data.coords, 
            attrs=fine_data.attrs
        )
        
        # Apply NaN mask
        downscaled_data_da = downscaled_data_da.where(~fine_nan_mask, np.nan)
        
        downscaled_data_list.append(downscaled_data_da)
        del downscaled_data_da
    
    # Concatenate along the 'number' dimension
    downscaled_data = xr.concat(downscaled_data_list, dim='number')
    downscaled_data = downscaled_data.assign_coords(number=np.arange(len(downscaled_data_list)))
    downscaled_data = downscaled_data.transpose('time', 'lat', 'lon', 'number')
    del downscaled_data_list
    
    return downscaled_data

In [16]:
%%time
downscaled_data = generate_downscaled_data(generator, coarse_data, fine_data, fine_nan_mask, num_members=30)

2024-08-29 15:47:43.035267: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape ingenerator/conv_4_dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-08-29 15:47:43.419025: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-08-29 15:47:43.422535: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-08-29 15:47:43.432304: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-08-29 15:47:43.441747: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-08-29 15:47:43.680905: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-08-29 15:47:44.879810: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn chi

263/263 [==============================] - 4s 13ms/step
CPU times: user 2min 52s, sys: 51.3 s, total: 3min 44s
Wall time: 2min 57s


In [17]:
diro = '/work/bb0983/athul_satheesh/downscaled_data/europe/downscaled_data/'
downscaled_data.to_netcdf(f'{diro}e_obs_eu_downscaled_wgan_prob_extended_eu_{epoch}epochs_adamV5.nc')

In [18]:
fine_data = fine_data.where(~fine_nan_mask, np.nan)
# coarse_data = coarse_data.where(~coarse_nan_mask, np.nan)

In [ ]:
# print(f"CRPS : {np.nanmean(xs.crps_ensemble(observations=fine_data, forecasts=downscaled_data, member_dim='number') ):.4f}")
print(f"MAE1 : {np.nanmean(abs(fine_data - downscaled_data.sel(number=0)) ):.4f}")
print(f"MAE5: {np.nanmean(abs(fine_data - downscaled_data.sel(number=5)) ):.4f}")
print(f"MAE10: {np.nanmean(abs(fine_data - downscaled_data.sel(number=10)) ):.4f}")
print(f"MAE  : {np.nanmean(abs(fine_data - downscaled_data.mean('number')) ):.4f}")

MAE1 : 1.1187
MAE5: 1.1193
MAE10: 1.1182


In [ ]:
# date1 = '2014-06-09' #'2021-06-15'
# date2 = '2014-06-09' #'2021-09-15'

# crps_data = xs.crps_ensemble(observations=fine_data.sel(time=slice(date1, date2)), 
#                              forecasts=downscaled_data.sel(time=slice(date1, date2)),
#                             member_dim='number', dim='time')
# # Convert to xarray DataArray if necessary
# crps_data = xr.DataArray(crps_data, dims=["lat", "lon"], coords={"lat": fine_data.lat, 
#                                                                          "lon": fine_data.lon,
#                                                                         })

# # Define the colormap colors
# colors_blues2black = [(0, 0, 0), (0, 0.90, 1)]  # Blue to Black
# colors_RdBlBu = [(1, 0, 0), (0, 0, 0), (0, 0.90, 1)]  # Blue to Black

# # Create the colormap
# cmap_name = 'BluesToBlack'
# blues_to_black = mcolors.LinearSegmentedColormap.from_list(cmap_name, colors_blues2black)
# Rd_bl_Bu = mcolors.LinearSegmentedColormap.from_list('RdBlBu', colors_RdBlBu)


# fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(12,10), sharex=True, sharey=True,
#                         subplot_kw=dict(projection=ccrs.PlateCarree(), facecolor='black'),
#                         gridspec_kw={'wspace': -0.05, 'hspace': -0.45})

# # Define the plots list
# plots = [coarse_data, fine_data, downscaled_data.mean(dim='number'), crps_data]
# # cmap = [blues_to_black, blues_to_black, blues_to_black, 'afmhot']
# cmap = ['turbo', 'turbo', 'turbo', 'afmhot']
# levels = [np.arange(0,20.125,0.125), np.arange(0,20.125,0.125), np.arange(0,20.125,0.125), np.arange(0,10.125,0.125)]
# title = ['a) Coarse','b) Fine','c) Downscaled ENS mean','d) CRPS']
# ticks = [np.arange(0,24,4), np.arange(0,24,4), np.arange(0,24,4), np.arange(0,12,2)]

# for i, ax in enumerate(axs.flatten()):
#     if i != (len(plots) - 1):
#         plots[i].sel(time=slice(date1, date2)).mean('time').plot(cmap=cmap[i], levels=levels[i], 
#                                                                  ax=ax, cbar_kwargs={'orientation':'vertical',
#                                                                                     'pad':0.01, 'label':'',
#                                                                                     'shrink':0.48, 'drawedges':False,
#                                                                                     'ticks': ticks[i], },
#                                                                  alpha=0.75)
#     else:
#         plots[-1].plot(cmap=cmap[i], levels=levels[i], 
#                        ax=ax, cbar_kwargs={'orientation':'vertical',
#                                            'pad':0.01, 'label':'',
#                                            'shrink':0.48, 'drawedges':False,
#                                            'ticks': ticks[i], },
#                        alpha=0.90)
    
#     ax.text(0, 55.2, f'{title[i]}', size=13, color='white')
#     ax.coastlines(linewidth=1.5, color='white')
#     ax.add_feature(cfeature.BORDERS.with_scale('50m'), linewidth=1.5, edgecolor='white')
#     ax.patch.set_facecolor('black')

# plt.show()